<a href="https://colab.research.google.com/github/dtabuena/Workshop/blob/main/RNA_Workshop/KZBG_edited.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install scanpy
!pip install pybiomart
!pip install python-igraph
!pip install louvain
!pip install pynndescent
from IPython.display import clear_output
clear_output()


In [15]:
import os
import h5py
import numpy as np
import scipy as sci
import matplotlib as mpl
from matplotlib import pyplot as plt
import scanpy as sc
import tarfile
import os
import anndata as ad
import pandas as pd
import pybiomart
from tqdm import tqdm
from matplotlib.pyplot import rc_context


In [16]:
os.chdir("C:/Users/dennis.tabuena/Dropbox (Gladstone)/0_Projects/_ReAnalyze_Zalocusky_2021")

def publishable_plots(FS=6):
    plt.rcParams.update({'font.size': FS,'axes.linewidth':.5,'figure.dpi':300,
                         'xtick.major.width': 0.5,'ytick.major.width': 0.5,
                         'figure.titlesize':FS,'axes.titlesize': FS,'xtick.labelsize': FS,
                         'ytick.labelsize':FS,'axes.labelsize': FS,'legend.fontsize': FS,
                         'figure.labelsize':FS})

    import urllib.request
    arial_link = 'https://raw.githubusercontent.com/dtabuena/Resources/main/Fonts/arial.ttf'
    filename = './arial.ttf'
    urllib.request.urlretrieve(arial_link, filename)
    plt.rcParams.update({'font.family': 'arial'})
    return None


def seurat_cmap(bitdepth=256,orig=11,s=.65,v=1,alpha=1):
    orig=orig/360
    bits = np.linspace(orig,orig+1,num=bitdepth+3)
    colors = [list(mpl.colors.hsv_to_rgb([h, s, v]))+[alpha] for h in bits]
    return colors


publishable_plots()
sc.settings.set_figure_params(dpi=300, facecolor='white',fontsize=6,)

In [17]:
os.chdir("C:/Users/dennis.tabuena/Dropbox (Gladstone)/0_Projects/Seurat_Scanpy/Scanpy_data/")
filename ='./zalocusky_2021_mouse_ApoE (1).h5ad'
kzbg_apoe_adata = sc.read_h5ad(filename)
display(kzbg_apoe_adata)
keys = list(np.arange(27))
seurat_id_order=[1,2,3,4,21,8,6,10,15,16,13,7,5,17,19,11,9,12,26,23,25,22,27,14,24,20,18]
seurat_id_order = [str(s) for s in seurat_id_order]
seurat_id_dict = { keys[i]:seurat_id_order[i] for i in range(len(seurat_id_order))   }
kzbg_apoe_adata.obs['res.0.6'] = [seurat_id_dict[int(c)] for c in  kzbg_apoe_adata.obs['res.0.6']]

AnnData object with n_obs × n_vars = 123489 × 21204
    obs: 'nGene', 'nUMI', 'orig.ident', 'sampleNumber', 'percent.mito', 'res.0.6', 'nCount_RNA', 'nFeature_RNA', 'group_vec'
    var: 'highly_variable'
    uns: 'pca'
    obsm: 'X_pca', 'X_tsne'
    varm: 'PCs'
    layers: 'counts', 'data'

In [18]:
display(kzbg_apoe_adata.var.head())

,highly_variable
Xkr4,0
Rp1,0
Sox17,0
Mrpl15,0
Lypla1,0


In [19]:
kzbg_apoe_adata.obs['group_vec']

AAACCTGAGAATGTTG-1     c(6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,...
AAACCTGAGACTAAGT-1     c(6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,...
AAACCTGAGGTGCACA-1     c(6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,...
AAACCTGCACCCATGG-1     c(6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,...
AAACCTGGTACGCTGC-1     c(6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,...
                                             ...                        
TTTGTCATCAATCACG-32    c(6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,...
TTTGTCATCGGTGTTA-32    c(6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,...
TTTGTCATCTACTTAC-32    c(6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,...
TTTGTCATCTGCGTAA-32    c(6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,...
TTTGTCATCTGGTTCC-32    c(6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,...
Name: group_vec, Length: 123489, dtype: object

In [20]:
age_bin_dict = {"5mo":'05mo', "10mo":'10mo', "14mo":'15_20m', "15mo":'15_20mo', "20mo":'15_20mo'}
kzbg_apoe_adata.obs['geno'] = [grp.split('_')[0] for grp in kzbg_apoe_adata.obs['group_vec']]
kzbg_apoe_adata.obs['age_bin'] = [age_bin_dict[grp.split('_')[1]] for grp in kzbg_apoe_adata.obs['group_vec']]


IndexError: ignored

In [ ]:

fig,ax =plt.subplots(1,figsize=[2,2])
sc.pl.tsne(kzbg_apoe_adata,color=['res.0.6'],palette=seurat_cmap(27,s=.65,v=1),ax=ax) # ,legend_loc='on data'


In [ ]:
display(kzbg_apoe_adata.obs)
kzbg_KO_adata.obs

In [ ]:
os.chdir("C:/Users/dennis.tabuena/Dropbox (Gladstone)/0_Projects/Seurat_Scanpy/Scanpy_data/")
filename ='./zalocusky_2021_mouse_ApoE-KO.h5ad'
kzbg_KO_adata = sc.read_h5ad(filename)
display(kzbg_KO_adata)
fig,ax =plt.subplots(1,figsize=[2,2])
sc.pl.tsne(kzbg_KO_adata,color='res.0.6',palette=seurat_cmap(len(set(kzbg_KO_adata.obs['res.0.6'])),s=.65,v=1),ax=ax)

In [ ]:
print(set( kzbg_KO_adata.obs['KO']))
print(set( kzbg_KO_adata.obs['animal_ID']))
print(set( kzbg_KO_adata.obs['Apoe']))
print(set( kzbg_KO_adata.obs['Sample.Description']))
print(set( kzbg_KO_adata.obs['Sample']))

In [ ]:
# sc.tl.pca(kzbg_KO_adata, svd_solver='arpack')
# sc.tl.pca(kzbg_apoe_adata, svd_solver='arpack')


In [ ]:
# overlap_var_names = kzbg_apoe_adata.var_names.intersection(kzbg_KO_adata.var_names)
# print(overlap_var_names)
# kzbg_apoe_adata = kzbg_apoe_adata[:, overlap_var_names]
# kzbg_KO_adata = kzbg_KO_adata[:, overlap_var_names]

In [ ]:
# sc.pp.neighbors(kzbg_KO_adata, n_neighbors=10, n_pcs=40)
# sc.pp.neighbors(kzbg_apoe_adata, n_neighbors=10, n_pcs=40)

In [ ]:
# display(kzbg_KO_adata.obs.head(5))
#
